### Install sklearn for Windows devices

In [ ]:
!pip install -U scikit-learn

### Install sklearn for MacOS

In [ ]:
!pip3 install -U scikit-learn

## Load Data

In [12]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv('trg.csv')
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(data.abstract)

data = data.rename(columns={'class': 'class_label'})
X = data.drop('class_label', axis=1)
y = data['class_label']
data  = pd.DataFrame(data= matrix.toarray(), columns = vectorizer.get_feature_names_out())
data.head()

,00,000,0001,00017,0003,0004,0005,001,0021,003,...,zwf1,zwitterionic,zygomycete,zygomycetes,zygotic,zymogen,zymogens,zymogram,zymography,zymomonas
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Standard Naive Bayes Model

In [52]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X['abstract'], y, test_size=0.1, random_state=42)

class NaiveBayes:
    def __init__(self):
        self.word_counts = defaultdict(lambda: defaultdict(int))
        self.class_counts = defaultdict(int)
        self.vocab = set()
        self.class_priors = {}

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.vocab = set()
        for cls in self.classes:
            self.class_counts[cls] = sum(1 for label in y if label == cls)
        for abstract, label in zip(X, y):
            for word in set(abstract.split()):
                self.word_counts[label][word] += 1
                self.vocab.add(word)
        self.vocab_size = len(self.vocab)
        total_docs = len(X)
        self.class_priors = {cls: self.class_counts[cls] / total_docs for cls in self.classes}

    def predict(self, documents):
        predictions = []
        for document in documents:
            posterior_probs = {cls: np.log(self.class_priors[cls]) for cls in self.classes}
            document_words = document.split()
            word_counts = {word: document_words.count(word) for word in set(document_words)}
            for word, count in word_counts.items():
                for cls in self.classes:
                    word_count_in_class = self.word_counts[cls].get(word, 0)
                    total_words_in_class = sum(self.word_counts[cls].values())
                    conditional_prob = (word_count_in_class + 1) / (total_words_in_class + self.vocab_size)
                    posterior_probs[cls] += count * np.log(conditional_prob)
            predicted_class = max(posterior_probs, key=posterior_probs.get)
            predictions.append(predicted_class)
        return predictions

nb_clf = NaiveBayes()
nb_clf.fit(X_train, y_train)
y_pred = nb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.93


# Improved with extra-preprocessing & Evaluating model's performance using k-fold cross validation

In [13]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

X_train, X_test, y_train, y_test = train_test_split(X['abstract'], y, test_size=0.01, random_state=42)

class NaiveBayes:
    def __init__(self, alpha=1.0):
        self.word_counts = defaultdict(lambda: defaultdict(int))
        self.class_counts = defaultdict(int)
        self.vocab = set()
        self.class_priors = {}
        self.alpha = alpha

    def preprocess(self, text):
        words = text.split()
        processed_words = []
        skip_next = False
        skip_next_next = False

        for i, word in enumerate(words):
            if skip_next:
                skip_next = False
                if skip_next_next:
                    skip_next = True
                    skip_next_next = False
                continue

            if word == "homo" and i + 1 < len(words) and words[i + 1] == "sapiens":
                processed_words.append("homo-sapiens")
                skip_next = True
                continue

            if word == "escherichia" and i + 1 < len(words) and words[i + 1] == "coli":
                processed_words.append("escherichia-coli")
                skip_next = True
                continue

            if word == "human" and i + 2 < len(words) and words[i + 1] == "immunodeficiency" and words[i + 2] == "virus":
                processed_words.append("human-immunodeficiency-virus")
                skip_next = True
                skip_next_next = True

            if word not in ENGLISH_STOP_WORDS:
                processed_words.append(word)

        return " ".join(processed_words)

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.vocab = set()

        for cls in self.classes:
            self.class_counts[cls] = sum(1 for label in y if label == cls)

        for abstract, label in zip(X, y):
            abstract = self.preprocess(abstract)
            for word in set(abstract.split()):
                self.word_counts[label][word] += 1
                self.vocab.add(word)

        self.vocab_size = len(self.vocab)
        total_docs = len(X)
        self.class_priors = {cls: self.class_counts[cls] / total_docs for cls in self.classes}

    def predict(self, documents):
        predictions = []
        for document in documents:
            document = self.preprocess(document)
            posterior_probs = {cls: np.log(self.class_priors[cls]) for cls in self.classes}
            document_words = document.split()
            word_counts = {word: document_words.count(word) for word in set(document_words)}

            for word, count in word_counts.items():
                for cls in self.classes:
                    word_count_in_class = self.word_counts[cls].get(word, 0)
                    total_words_in_class = sum(self.word_counts[cls].values())
                    conditional_prob = (word_count_in_class + self.alpha) / (total_words_in_class + self.alpha * self.vocab_size)
                    posterior_probs[cls] += count * np.log(conditional_prob)

            predicted_class = max(posterior_probs, key=posterior_probs.get)
            predictions.append(predicted_class)

        return predictions

kf = KFold(n_splits=5, random_state=42, shuffle=True)
accuracies = []

nb_clf = NaiveBayes(alpha=0.5)
nb_clf.fit(X_train, y_train)
y_pred = nb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

for train_index, test_index in kf.split(X['abstract']):
    X_train, X_test = X['abstract'][train_index], X['abstract'][test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    nb_clf.fit(X_train, y_train)
    y_pred = nb_clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

print(f"5-fold cross-validation accuracy: {np.mean(accuracies):.2f}")

Accuracy: 0.97
5-fold cross-validation accuracy: 0.99


## Export tst.csv predictions for kaggle

In [11]:
data = pd.read_csv('tst.csv')

vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(data.abstract)
X = data.drop('id', axis=1)

y_pred = nb_clf.predict(X['abstract'])

df_predictions = pd.DataFrame({'id': range(1, len(y_pred) + 1), 'class': y_pred})
df_predictions.to_csv('predictions.csv', index=False)

## Report Section

explain and motivate the chosen representation & data preprocessing,


explain the idea behind the model improvements and their implementation (including the implementation of the standard Naive Bayes)

explain the evaluation procedure (e.g., cross-validation or training/validation split)

include and explain the training/validation results for the standard and improved Naive Bayes model. You can summarize results using tables (or plots), but all results have to be explained descriptively as well. be written in plain English and should not be longer than two A4 pages (export the notebook as pdf to see if the report section fits in two pages).